# Oracle for DMDGP

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DEL = 1e-4
EPS = 2 / 3

In [14]:
instance = pd.read_csv('6_15.csv')
instance

,u,v,duv
0,2,6,3.543775
1,1,5,3.626174
2,1,6,4.088372
3,1,2,1.526000
4,1,4,3.627452
5,3,6,3.389128
6,4,6,2.491390
7,2,5,2.578011
8,3,5,2.491390
9,3,4,1.526000


In [4]:
n = max(instance['v']) 
m = len(instance['duv']) 

g = [[] for _ in range(n + 1)]
edges = []
prune_edges = []
d = np.zeros((n + 1, n + 1))

for i, r in instance.iterrows():
    u = int(r['u'])
    v = int(r['v'])
    duv = r['duv']

    g[u].append((v, duv))
    g[v].append((u, duv))
    edges.append((u, v, duv))
    d[u][v] = duv
    d[v][u] = duv

    if u + 3 < v:
        prune_edges.append((u, v))

In [5]:
th = np.zeros(n + 1)
for k in range(3, n + 1):
    th[k] = np.arccos((d[k - 1][k]**2 + d[k - 2][k - 1]**2 - d[k - 2][k]**2) / (2 * d[k - 1][k] * d[k - 2][k - 1]))

In [6]:
cw = np.zeros(n + 1)
for k in range(4, n + 1):
    rij = d[k - 3][k - 2]
    rjk = d[k - 2][k - 1]
    rkl = d[k - 1][k]
    rjl = d[k - 2][k]
    ril = d[k - 3][k]
    st = np.sin(th[k - 1])
    ct = np.cos(th[k - 1])

    num = rij**2 + rjl**2 - ril**2 - rij * (rjl**2 + rjk**2 - rkl**2) * ct / rjk
    den = rij * np.sqrt(4 * rjl**2 * rjk**2 - (rjl**2 + rjk**2 - rkl**2)**2) * st / rjk
    cw[k] = num / den

In [7]:
def dist(xa, xb):
    return np.linalg.norm(xa - xb)


def toBin(k):
    return format(k, 'b').zfill(n - 3)

## Função $h$

Dado um cadidado a solução $k \in \left\{0, 1, \dots, 2^{n-3} - 1\right\}$, $h$ associa $k$ a uma realização $\mathbf{x}^k = \left( \mathbf{x}_{1}^{k}, \dots, \mathbf{x}_{n}^{k}  \right)$, onde cada $\mathbf{x}_{i}^{k}$ é dado por:

$$
\mathbf{x}_{i}^{k} = \left(\prod_{j=1}^{i} B_{j}\right) \begin{bmatrix}0\\0\\0\\1\end{bmatrix},
$$
onde
$$ B_1 = \begin{bmatrix}
			1 & 0 & 0 & 0\\ 
			0 & 1 & 0 & 0\\ 
			0 & 0 & 1 & 0\\ 
			0 & 0 & 0 & 1
		\end{bmatrix},  B_2 = \begin{bmatrix}
			-1 & 0 & 0 & -d_{1,2}\\ 
			0 & 1 & 0 & 0\\ 
			0 & 0 & -1 & 0\\ 
			0 & 0 & 0 & 1
		\end{bmatrix},
B_3 = \begin{bmatrix}
			-\cos \theta_{1,3} & -\sin\theta_{1,3}  & 0 & -d_{2,3} \cos \theta_{1,3} \\
			\sin \theta_{1,3}  & -\cos \theta_{1,3} & 0 & d_{2,3}\sin\theta_{1,3}    \\
			0                  & 0                  & 1 & 0                          \\
			0                  & 0                  & 0 & 1
		\end{bmatrix}$$ e $$ B_i = \begin{bmatrix}
			-\text{c}( \theta_i)                   & -\text{s}( \theta_i)                     & 0                       & -d_{i-1,i}\text{c}( \theta_i)                  \\
			\text{s}( \theta_i) \text{c}(\omega_i) & -\text{c}( \theta_i) \text{c}( \omega_i) & - \text{s}( \omega_i)   &  d_{i-1,i}\text{s}( \theta_i) \text{c}( \omega_i) \\
			\text{s}( \theta_i) \text{s}(\omega_i) & -\text{c}( \theta_i) \text{s}( \omega_i) &   \text{c}( \omega_i)   &  d_{i-1,i}\text{s}( \theta_i) \text{s}( \omega_i) \\
			0                                      & 0                                        & 0                       & 1
		\end{bmatrix} $$ onde
		$$\begin{matrix}
			\text{c}( \theta_i) & = & \cos\theta_{i-2,i}, & \, & \text{s}( \theta_i) & = & \sin\theta_{i-2,i} \\
			\text{c}( \omega_i) & = & \cos\omega_{i-3,i}, & \, & \text{s}( \omega_i) & = & \sin\omega_{i-3,i}.
		\end{matrix}		
		$$

O valor de $\sin\omega_{i-3, i} = \pm \sqrt{ 1 - \cos^2\omega_{i-3,i} }$ é definido a partir de $k$ em sua representação binária. 

In [8]:
def h(k):
    X = np.zeros((n + 1, 4))
    X[1] = np.array([0, 0, 0, 1])
    X[2] = np.array([-d[1][2], 0, 0, 1])
    X[3] = np.array([-d[1][2] + d[2][3] * np.cos(th[3]), d[2][3] * np.sin(th[3]), 0, 1])

    B2 = np.array([[-1, 0, 0, -d[1][2]],
                   [0, 1, 0, 0], 
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])
    ct = np.cos(th[3])
    st = np.sin(th[3])
    di = d[2][3]
    B3 = np.array([[-ct, -st, 0, -di * ct], 
                   [st, -ct, 0, di * st],
                   [0, 0, 1, 0], 
                   [0, 0, 0, 1]])
    B = B2 @ B3

    for i in range(4, n + 1):
        sgn = 1
        if k & (1 << (n - i)) == 0:
            sgn = 0

        ct, st = np.cos(th[i]), np.sin(th[i])
        di = d[i - 1][i]
        sw = (-1)**sgn * np.sqrt(1 - cw[i]**2)

        Bi = np.array([[-ct, -st, 0, -di * ct],
                       [st * cw[i], -ct * cw[i], -sw, di * st * cw[i]],
                       [st * sw, -ct * sw, cw[i], di * st * sw], 
                       [0, 0, 0, 1]])
        B = B @ Bi
        X[i] = B @ X[1]

    return X[:, :-1]

## Função $g$

Dada uma realização $\mathbf{x} = (\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_2)$, $g$ é definida como:

$$ g(\mathbf{x}) = \sum_{(u,v) \in E} \left( \left\lVert \mathbf{x}_u - \mathbf{x}_v \right\rVert^2   - d_{u,v}^2 \right)^2 $$

In [9]:
def g(X):
    return np.sum([(dist(X[e[0]], X[e[1]])**2 - e[2]**2)**2 for e in edges])

## Função $\varphi$

Seja $\varphi : I \subseteq \mathbb{R} \to [0,1]$ satisfazendo:

 - $\varphi$ é estritamente crescente;
 - $\varphi(\delta) = 1 - \varepsilon$.
 
Então
    $$ f(k) = 1 - \left\lfloor \varphi(g(h(k))) + \varepsilon \right\rfloor $$
define um oráculo para o DMDGP. 

In [10]:
def phi(xi, caso=0):
    if caso == 1:
        p1 = 6**4 * (n**6 + n**2)
        p2 = np.log(DEL / p1) / np.log(1 - EPS)
        return (xi / p1)**(1 / p2)
    if caso == 2:
        return np.arctan(xi - DEL + np.tan(np.pi * (1 / 2 - EPS))) / np.pi + 1 / 2

    return 1 / (1 + (EPS / (1 - EPS)) * np.exp(DEL - xi))

In [11]:
def f(k):
    xi = g(h(k))
    return int(1 - np.floor(phi(xi) + EPS))

## Encontrando a solução

In [13]:
for k in range(1 << (n - 3)):
    fk = f(k)
    print("f(%s) = %d" % (toBin(k), fk))

#     if fk == 1:
#         print(h(k)[1:])

f(000) = 0
f(001) = 0
f(010) = 0
f(011) = 1
f(100) = 1
f(101) = 0
f(110) = 0
f(111) = 0
